# RAG with OpenAI GPT Models

In this notebook I am going to build RAG system using OpenAI model. Learning how to extract and process data, create embeddings, buidling a RAG system and to generate the responses.
If you use this notebook, you will be able to turn any data into a smart assistant that retrieves and generates info effortlessly.

**Lerning objetives through this notebook:**

1. Data conversion mastery
> Transform PDF and Images into AI ready formats. This is the first critical step in making data usuable for advanced AI models
2. Advanced OCR with GPT
> Tecnology that reads pdfs
3. Building a Retrival System that works
4. Seamless integration of Retrival and Generation
5. Fine-Tuning with prompt Engineering

**By the end of the section you will be able:**

1. Convert and Prepare Complex Data
2. Extract rich, structured information
3. Create and Utilize Embeddings
> Represent your data in ways that can make it easy for AI model to Retrieve and compare
4. Build a Powerfull Retrival System
5. Integrate Retrival and Generation like apro
6. Optimize AI Outputs with Precision

**Things we need to consider:**

* RAG systems can struggle with ambiguous queries or complex data.

## Case Study - Cooking books

Cooking Books?

Cook books are full of valuable info, but finding what you need can be tricky!

Herecomes the perfect Solution --> **RAG**

We will start with PDFs and convert its content into a format our AI can use --> **Images**

Next, we will use GPT models to extract text from the iamges, focusing on structuring data.

On 3rd stage, we will create embeddings wich are essentialy a numerical representation of the data, this embeddings will allow our system to understand the context and relevance of each peace of indormation.

The we will integrate everything. You will create a system that not only find recipes but also answer questions.

Your AI will provide accurate, useful, and context-aware response.



And btw...you can apply these techniques to nearly any field. With these skills, you can convert any unstructured data into dynamic systems...

#Setup

In [1]:
from google.colab import userdata
openai_api_key = userdata.get('genai_course')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Ideas/GenAI/RAG/RAG with OpenAI

/content/drive/MyDrive/Ideas/GenAI/RAG/RAG with OpenAI


video = Converting PDF to Images

# Perform OCR and transform to images

In [4]:
!pip install pdf2image
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.11 [186 kB]
Fetched 186 kB in 0s (1,793 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126718 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.11_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.11) ...
Setting up poppler-utils (22.02.0-2ubuntu0.11) ...
Processing triggers for man-db (2.10.2-1) ...


In [5]:
# Import libraries
from pdf2image import convert_from_path
import os

In [6]:
# Create a function to converts pdfs into images and stores the paths
def pdf_to_images(pdf_path, output_folder):
  # If the path were we are going to save the images does not exists
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)

  # Convert PDF into images
  images = convert_from_path(pdf_path)
  image_paths = []

  # Save images and paths
  for i, image in enumerate(images):
    image_path = os.path.join(output_folder, f'page{i + 1}.jpg')
    image.save(image_path, 'JPEG')
    image_paths.append(image_path)

  return image_paths

For effiency porpuse of the cpu that google provides, I am just going to use an edited format from "Things mother used to make.pdf" wich it only has 4 pages instead of 130 pages.

In [7]:
# At the end, we want to have
pdf_path = "ed_Things_mother_used_to_make.pdf"
output_folder = 'images'
image_paths = pdf_to_images(pdf_path, output_folder)

In [ ]:
image_paths

['images/page1.jpg',
 'images/page2.jpg',
 'images/page3.jpg',
 'images/page4.jpg']

video = Reading a Single Image with GPT

We are going to use gpt to extract information from the images.

In [8]:
!pip install openai

In [9]:
# Import libraries
from openai import OpenAI
import base64 # for the images to be encoded

In [10]:
# 1. Setup connecting to openai API
client = OpenAI(
    api_key = openai_api_key
)
model = 'gpt-4o-mini'

In [11]:
# Read and encode one image
image_path = 'images/page2.jpg'
with open(image_path, 'rb') as image_file: # rb: readbinary (it will read in raw bytes without any text-based interpretation)
  image_data = base64.b64encode(image_file.read()).decode('utf-8')
image_data

'...'

In [12]:
# Define the system prompt. Let's keep it simplefor now
system_prompt = """
Please analyze the content of this image and extract any related recipe information.
"""

In [ ]:
# Call the OpenAI API use the chat completion method
response = client.chat.completions.create(
    model = model,
    messages = [
        # Provide the system prompt
        {'role': 'system', 'content': system_prompt},

        # The user message contains both the text and image URL / path
        {'role': 'user', 'content': [
            'This is the image from the recipe page.',
            {'type': 'image_url', 'image_url': {'url': f'data:image/jpeg;base64,{image_data}',
                                                'detail': 'low'}} # Image quality
        ]}
    ]
)

In [ ]:
# Display the content
response

ChatCompletion(id='chatcmpl-CTzDXt3aGtJxoBKAAq6FHQxuQ9hps', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Based on the text you've shared, here are the recipes contained in the image:\n\n### Bannocks\n**Ingredients:**\n- 1 Cupful of Thick Sour Milk\n- ½ Cupful of Sugar\n- 2 Cupfuls of Flour\n- ½ Cupful of Indian Meal\n- 1 Teaspoonful of Soda\n- A pinch of Salt\n\n**Instructions:**\n1. Make the mixture sit to drop from a spoon.\n2. Drop mixture, size of a walnut, into boiling fat.\n3. Serve warm with maple syrup.\n\n---\n\n### Boston Brown Bread\n**Ingredients:**\n- 1 Cupful of Rye Meal\n- 1 Cupful of Graham Meal\n- 1 Cupful of Flour\n- 1 Cupful of Sour Milk\n- 1 Cupful of Molasses\n- 1 Teaspoonful of Salt\n- 1 Heaping Teaspoonful of Baking Soda\n- ½ Cupful of Sweet Milk\n\n**Instructions:**\n1. Stir the meals and salt together.\n2. Beat the soda into the molasses until it foams.\n3. Add sour milk, mix well, and pour into a tin pan 

In [ ]:
# Display the content
gpt_response = response.choices[0].message.content
gpt_response

"Based on the text you've shared, here are the recipes contained in the image:\n\n### Bannocks\n**Ingredients:**\n- 1 Cupful of Thick Sour Milk\n- ½ Cupful of Sugar\n- 2 Cupfuls of Flour\n- ½ Cupful of Indian Meal\n- 1 Teaspoonful of Soda\n- A pinch of Salt\n\n**Instructions:**\n1. Make the mixture sit to drop from a spoon.\n2. Drop mixture, size of a walnut, into boiling fat.\n3. Serve warm with maple syrup.\n\n---\n\n### Boston Brown Bread\n**Ingredients:**\n- 1 Cupful of Rye Meal\n- 1 Cupful of Graham Meal\n- 1 Cupful of Flour\n- 1 Cupful of Sour Milk\n- 1 Cupful of Molasses\n- 1 Teaspoonful of Salt\n- 1 Heaping Teaspoonful of Baking Soda\n- ½ Cupful of Sweet Milk\n\n**Instructions:**\n1. Stir the meals and salt together.\n2. Beat the soda into the molasses until it foams.\n3. Add sour milk, mix well, and pour into a tin pan which has been well greased.\n4. If you are using a brown-bread steamer, there is no need for extra greasing.\n\nFeel free to ask if you need more information o

In [ ]:
from IPython.display import Markdown, display
display(Markdown(gpt_response))

Based on the text you've shared, here are the recipes contained in the image:

### Bannocks
**Ingredients:**
- 1 Cupful of Thick Sour Milk
- ½ Cupful of Sugar
- 2 Cupfuls of Flour
- ½ Cupful of Indian Meal
- 1 Teaspoonful of Soda
- A pinch of Salt

**Instructions:**
1. Make the mixture sit to drop from a spoon.
2. Drop mixture, size of a walnut, into boiling fat.
3. Serve warm with maple syrup.

---

### Boston Brown Bread
**Ingredients:**
- 1 Cupful of Rye Meal
- 1 Cupful of Graham Meal
- 1 Cupful of Flour
- 1 Cupful of Sour Milk
- 1 Cupful of Molasses
- 1 Teaspoonful of Salt
- 1 Heaping Teaspoonful of Baking Soda
- ½ Cupful of Sweet Milk

**Instructions:**
1. Stir the meals and salt together.
2. Beat the soda into the molasses until it foams.
3. Add sour milk, mix well, and pour into a tin pan which has been well greased.
4. If you are using a brown-bread steamer, there is no need for extra greasing.

Feel free to ask if you need more information or details!

video = Enhancing AI with Prompt Engineering

Trying to improve the response using Prompt engineering

In [95]:
# Define a function to get the gpt response and display in markdown
def get_gpt_response(response):
  gpt_response = response.choices[0].message.content
  return display(Markdown(gpt_response))

In [14]:
# Define improved system prompt
system_prompt2 = """
Please analyze the content of this image and extract any related recipe information into structure components.
Specifically, extra recipe title, list of ingredients, step by step instructions, cuisine type, dish type, any relevant tags or metadata.
The output must be formatted in a way suited for embedding in a Retrival Augmented Generation (RAG) system.
If you see a page with a table of contents response with 'None'.
"""

In [ ]:
# Call the API to extract the information
response = client.chat.completions.create(
    model = model,
    messages = [
        {'role': 'system', 'content': system_prompt2},
        {'role': 'user', 'content':[
            'This is the image from the recipe page',
            {'type': 'image_url',
             'image_url': {'url': f'data:image/jepg;base64,{image_data}',
                           'detail': 'low'}}
        ]}
    ],
    temperature = 0, # No creative required here
)

In [ ]:
# Print the info from the page with the improved prompt
get_gpt_response()

Based on the content of the image, here is the structured recipe information:

### Recipe Title
Corned Beef

### Ingredients
- Corned beef (specific quantity not provided)

### Instructions
1. Should boil for four hours.

### Cuisine Type
Not specified

### Dish Type
Main dish

### Relevant Tags/Metadata
- Cooking time: 4 hours
- Method: Boiling

---

If you need further assistance or additional details, feel free to ask!

video = Reading All Images in a Dataset

I am only going to use 3

In [15]:
image_paths

['images/page1.jpg',
 'images/page2.jpg',
 'images/page3.jpg',
 'images/page4.jpg']

In [ ]:
# Extract the info about all of the images / recipes
extracted_recipes = []

for image_path in image_paths:
  print(f'Processing image {image_path}')
  # Reading and decoding images
  with open(image_path, 'rb') as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

  # Call the API to extract the information
  response = client.chat.completions.create(
      model = model,
      messages = [
          {'role': 'system', 'content': system_prompt2},
          {'role': 'user', 'content':[
              'This is the image from the recipe page',
              {'type': 'image_url',
              'image_url': {'url': f'data:image/jpeg;base64,{image_data}',
                            'detail': 'low'}}
          ]}
      ],
      temperature = 0, # No creative required here
  )

  # Extract the content and store it
  gpt_response = response.choices[0].message.content
  extracted_recipes.append({'image_path': image_path, 'recipe_info': gpt_response})
  print(f'Extracted information for {image_path}:\n{gpt_response}\n')

Processing image images/page1.jpg
Extracted information for images/page1.jpg:
None

Processing image images/page2.jpg
Extracted information for images/page2.jpg:
Here’s the structured information extracted from the recipe image:

### Recipe Title
Things Mother Used to Make - Breads

### Ingredients
#### Bannocks
- 1 Cupful of Thick Sour Milk
- ½ Cupful of Sugar
- 2 Cupfuls of Flour
- ½ Cupful of Indian Meal
- 1 Teaspoonful of Soda
- A pinch of Salt

#### Boston Brown Bread
- 1 Cupful of Rye Meal
- 1 Cupful of Graham Meal
- 1 Cupful of Flour
- 1 Cupful of Sour Milk
- 1 Cupful of Molasses
- ½ Teaspoonful of Salt
- 1 Heaping Teaspoonful of Soda
- 1 Cupful of Sweet Milk

### Step-by-Step Instructions
1. For Bannocks: Make the mixture stiff enough to drop from a spoon. Drop mixture, size of a walnut, into boiling fat. Serve warm, with maple syrup.
2. For Boston Brown Bread: Stir the meals and salt together. Beat the soda into the molasses until it foams; add sour milk, mix well, and pour in

video = Filtering Non-relevant Information

In [42]:
# Filter out non-recipe content based on key-recipe related terms
filtered_recipes = []
for recipe in extracted_recipes:
  if any(keyword in recipe['recipe_info'].lower() for keyword in ['ingredients',
                                                                  'instructions',
                                                                  'recipe title']):
    filtered_recipes.append(recipe)
    print(f'Added recipe: {recipe['image_path']}')
  else:
    print(f'Skipping recipe: {recipe['image_path']}')

NameError: name 'extracted_recipes' is not defined

video = Understanding Embeddings in NLP

video = Generating Embeddings

In [18]:
import json

In [ ]:
# Define the output file path
# we are going to save our filtererd recipes so we dont need to go through it all over again
output_file = 'recipe_info.json'

# Write the filtered list to a jason file
with open(output_file, 'w') as json_file:
  json.dump(filtered_recipes, json_file, indent = 4)

# Embeddings

Embeddings turn raw text into numerical representations that machines can easily process.

Embeddings are dense vector representations of text that turns words into high-dimensional numbers. These vectors capture meaning, helping AI models undestand text relationships.

Building Embeddings: Step by step:

1. Text preparation
> We took the instructions of the recipe and then tokenize the text, then...
2. Embedding generation
> Convert each tokenize of text into a vector
3. Undestanding Vector Space
> We will that there are vector closer from each pther
4. Using Embeddings for Retrival

In [16]:
# import libraries
import numpy as np

In [49]:
# Load the filtered recipes
with open('recipe_info.json', 'r') as json_file:
  filtered_recipes = json.load(json_file)

In [50]:
filtered_recipes

[{'image_path': 'images/page2.jpg',
  'recipe_info': 'Here’s the structured information extracted from the recipe image:\n\n### Recipe Title\nThings Mother Used to Make - Breads\n\n### Ingredients\n#### Bannocks\n- 1 Cupful of Thick Sour Milk\n- ½ Cupful of Sugar\n- 2 Cupfuls of Flour\n- ½ Cupful of Indian Meal\n- 1 Teaspoonful of Soda\n- A pinch of Salt\n\n#### Boston Brown Bread\n- 1 Cupful of Rye Meal\n- 1 Cupful of Graham Meal\n- 1 Cupful of Flour\n- 1 Cupful of Sour Milk\n- 1 Cupful of Molasses\n- ½ Teaspoonful of Salt\n- 1 Heaping Teaspoonful of Soda\n- 1 Cupful of Sweet Milk\n\n### Step-by-Step Instructions\n1. For Bannocks: Make the mixture stiff enough to drop from a spoon. Drop mixture, size of a walnut, into boiling fat. Serve warm, with maple syrup.\n2. For Boston Brown Bread: Stir the meals and salt together. Beat the soda into the molasses until it foams; add sour milk, mix well, and pour into a tin pan which has been well greased, if you have no brown-bread steamer.\n\n#

In [51]:
# Generate embedding for each recipe info
# doc for embedding https://platform.openai.com/docs/guides/embeddings
recipe_texts = [recipe['recipe_info'] for recipe in filtered_recipes]
embedding_response = client.embeddings.create(
    input = recipe_texts,
    model = 'text-embedding-3-large'
)

Another options would be to organize per recipe, but it should be done in the preprocessing at prompt!

In [53]:
embedding_response.data

[Embedding(embedding=[0.01405092142522335, -0.029697859659790993, -0.015527610667049885, 0.0017106847371906042, -0.016273412853479385, -0.048029687255620956, 0.009061501361429691, -0.019241707399487495, -0.0007849572575651109, 0.04561328887939453, 0.0057725124061107635, 0.0009592886199243367, 0.007405819837003946, 0.010418862104415894, 0.00025194144109264016, -0.007383445743471384, 0.03391910344362259, 0.011709101498126984, -0.03651449456810951, -0.012701018713414669, -0.0004810426908079535, -0.002190795261412859, 0.04242125153541565, -0.02404467575252056, 0.032188840210437775, 0.006913590244948864, -0.046657413244247437, 0.006055917125195265, 0.004993148613721132, 0.014625188894569874, 0.01425228826701641, -0.003971398808062077, -0.016571734100580215, -0.007868217304348946, -0.025252876803278923, 0.0034922207705676556, 0.01231320109218359, 0.07690716534852982, 0.03036908246576786, 0.01948036439716816, 0.010933466255664825, 0.015990007668733597, -0.0026755668222904205, 0.00544063001871

In [52]:
# Extract the embeddings
embeddings = [data.embedding for data in embedding_response.data]
len(embeddings), embeddings

(3,
 [[0.01405092142522335,
   -0.029697859659790993,
   -0.015527610667049885,
   0.0017106847371906042,
   -0.016273412853479385,
   -0.048029687255620956,
   0.009061501361429691,
   -0.019241707399487495,
   -0.0007849572575651109,
   0.04561328887939453,
   0.0057725124061107635,
   0.0009592886199243367,
   0.007405819837003946,
   0.010418862104415894,
   0.00025194144109264016,
   -0.007383445743471384,
   0.03391910344362259,
   0.011709101498126984,
   -0.03651449456810951,
   -0.012701018713414669,
   -0.0004810426908079535,
   -0.002190795261412859,
   0.04242125153541565,
   -0.02404467575252056,
   0.032188840210437775,
   0.006913590244948864,
   -0.046657413244247437,
   0.006055917125195265,
   0.004993148613721132,
   0.014625188894569874,
   0.01425228826701641,
   -0.003971398808062077,
   -0.016571734100580215,
   -0.007868217304348946,
   -0.025252876803278923,
   0.0034922207705676556,
   0.01231320109218359,
   0.07690716534852982,
   0.03036908246576786,
   0.0

In [54]:
# Convert the embeddings to numpy array
embedding_matrix = np.array(embeddings)
embedding_matrix

array([[ 0.01405092, -0.02969786, -0.01552761, ...,  0.00205096,
        -0.01910746,  0.00858419],
       [-0.00078257, -0.02344496, -0.02162798, ..., -0.00067725,
        -0.01909299,  0.01057954],
       [-0.00925107, -0.0301129 , -0.01327998, ..., -0.00155022,
         0.00376076, -0.00436744]])

In [55]:
# Verify the embedding matrix
print(f'Generated embeddings for {len(filtered_recipes)} recipes. There are {len(filtered_recipes)} embeddings')
print(f'Each embedding is of size {len(embeddings[0])}')

Generated embeddings for 3 recipes. There are 3 embeddings
Each embedding is of size 3072


Each time we retrieve information we may get different results

video = Building FAISS Index and Metadata Integration

# Retrival System

In [27]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 56.6 MB/s eta 0:00:00


Uso en RAG (Retrieval-Augmented Generation)
FAISS es el corazón de la fase de Recuperación en un sistema RAG:

**Indexación**: Los documentos de tu base de conocimiento se convierten en embeddings vectoriales y se almacenan en un índice FAISS.

**Consulta**: Cuando el usuario hace una pregunta, esta se convierte en un embedding de consulta.

**Recuperación Rápida**: FAISS busca instantáneamente en su índice los 5 o 10 embeddings más cercanos. Estos corresponden a los fragmentos de texto más semánticamente relevantes para la pregunta, que luego se envían al LLM como contexto.

In [56]:
import faiss

In [57]:
# Print the embedding matrix shape
print(f'Embedding matrix shape: {embedding_matrix.shape}')

Embedding matrix shape: (3, 3072)


In [58]:
embedding_matrix.shape[1]

3072

In [59]:
# Initialize the FAISS index
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)

1. Inicialización del Índice (Línea 1)
Python

index = faiss.IndexFlatL2(embedding_matrix.shape[1])
Esta línea crea el objeto índice FAISS, que es la estructura de datos optimizada para la búsqueda rápida.

**faiss.IndexFlatL2:** Especifica el tipo de índice a usar.

>Flat indica que se utilizará el método de fuerza bruta (brute force), lo que significa que la búsqueda comparará el vector de consulta con todos los vectores almacenados. Aunque es el más lento, garantiza una precisión del 100% (exact nearest neighbor). Es común para conjuntos de datos pequeños o medianos.

>L2 se refiere a la métrica de distancia utilizada para medir la similitud: la distancia euclidiana (o norma L2). Cuanto menor sea la distancia L2 entre dos vectores, más similares son.

>embedding_matrix.shape[1]: Este argumento le dice a FAISS la dimensión de los vectores que va a almacenar.

>embedding_matrix es la matriz NumPy o similar que contiene todos tus embeddings (vectores).

>shape[1] accede a la segunda dimensión de la matriz (la longitud de cada vector individual, por ejemplo, 768 o 1024).

2. Carga de los Vectores (Línea 2)
Python

**index.add(embedding_matrix)**
Esta línea toma todos los embeddings que has generado a partir de tus documentos y los agrega a la estructura de índice que acabas de crear.

>embedding_matrix: Es la matriz que contiene todos los vectores que representan tu base de conocimiento. Cada fila de esta matriz es un embedding de un fragmento de texto.

>index.add(...): FAISS procesa estos vectores y los organiza internamente para futuras búsquedas. Después de esta línea, el índice está listo para ser consultado.

In [60]:
# Save the index
faiss.write_index(index, 'filtered_recipe_index.index')

En términos sencillos, hace lo siguiente:

Guarda el Índice: Toma el objeto de índice FAISS (index) que ya creaste y cargaste con todos tus embeddings (los vectores que representan tu conocimiento).

Escribe en Disco: Guarda la estructura de datos interna de FAISS en un archivo binario en tu sistema de archivos.

Detalle de la Operación
faiss.write_index(index, 'filtered_recipe_index.index'):

index: Es la estructura de datos que contiene todos los vectores de tu matriz (embedding_matrix) y la lógica de búsqueda (IndexFlatL2).

'filtered_recipe_index.index': Es el nombre del archivo donde se guardará el índice. La extensión .index es una convención común para los archivos binarios de FAISS.

In [61]:
recipe['recipe_info']

NameError: name 'recipe' is not defined

In [62]:
# Save the metadata
metadata = [{'recipe_info': recipe['recipe_info'],
             'image_path': recipe['image_path']} for recipe in filtered_recipes]
with open('recipe_metadada.json', 'w') as json_file:
  json.dump(metadata, json_file, indent = 4)

In [63]:
metadata

[{'recipe_info': 'Here’s the structured information extracted from the recipe image:\n\n### Recipe Title\nThings Mother Used to Make - Breads\n\n### Ingredients\n#### Bannocks\n- 1 Cupful of Thick Sour Milk\n- ½ Cupful of Sugar\n- 2 Cupfuls of Flour\n- ½ Cupful of Indian Meal\n- 1 Teaspoonful of Soda\n- A pinch of Salt\n\n#### Boston Brown Bread\n- 1 Cupful of Rye Meal\n- 1 Cupful of Graham Meal\n- 1 Cupful of Flour\n- 1 Cupful of Sour Milk\n- 1 Cupful of Molasses\n- ½ Teaspoonful of Salt\n- 1 Heaping Teaspoonful of Soda\n- 1 Cupful of Sweet Milk\n\n### Step-by-Step Instructions\n1. For Bannocks: Make the mixture stiff enough to drop from a spoon. Drop mixture, size of a walnut, into boiling fat. Serve warm, with maple syrup.\n2. For Boston Brown Bread: Stir the meals and salt together. Beat the soda into the molasses until it foams; add sour milk, mix well, and pour into a tin pan which has been well greased, if you have no brown-bread steamer.\n\n### Cuisine Type\nTraditional America

video = Implementing a Robust Retrieval System

WIP

In [64]:
# Generate the embeddings for the query
k = 5
query = 'How tp make bread?'
query_embedding = client.embeddings.create(
    input = [query],
    model = 'text-embedding-3-large'
).data[0].embedding
print(f'The query embedding is {query_embedding}\n')
query_vector = np.array(query_embedding).reshape(1, -1)
print(f'The query vector is {query_vector}\n')

The query embedding is [-0.01561812125146389, -0.03618989512324333, -0.006917146500200033, 0.014694123528897762, -0.02567942440509796, -0.040886882692575455, 0.021200604736804962, 0.01804361492395401, 0.01175529882311821, 0.009766138158738613, -0.004902319051325321, -0.020430607721209526, 0.00026348361279815435, -0.007693560793995857, 0.0282589178532362, -0.028849249705672264, 0.018967611715197563, -0.011864381842315197, 0.009689138270914555, 0.0007904508383944631, -0.021701103076338768, 0.012300713919103146, 0.010606719180941582, -0.026462256908416748, -0.007083979435265064, 0.001098048873245716, 0.011152134276926517, -0.010901885107159615, -0.02262510173022747, 0.043119873851537704, -0.00883572455495596, 0.030286578461527824, -0.012473964132368565, -0.012756296433508396, -0.021508604288101196, -0.005370734259486198, 0.00570119172334671, 0.007866810075938702, -0.013988292776048183, 0.001471818657591939, -0.016298286616802216, -0.0060605239123106, -0.007757727522403002, 0.0004579882370

In [65]:
# Search the FAISS index
distances, indices = index.search(query_vector, min(k, len(metadata)))
print(f'The distances are {distances}\n')
print(f'The indices are {indices}\n')

The distances are [[1.2459004 1.27947   1.5886095]]

The indices are [[0 1 2]]



In [84]:
# Define a function to query the embeddings
def query_embeddings(query, index, metadata, k = 5): # index: El índice FAISS pre-cargado con todos los embeddings.
  # Generate the embeddings for the query
  query_embedding = client.embeddings.create(
      input = [query],
      model = 'text-embedding-3-large'
  ).data[0].embedding

  print(f'The query embedding is {query_embedding}\n')
  query_vector = np.array(query_embedding).reshape(1, -1)
  print(f'The query vector is {query_vector}\n')
  print(f'Query shape after reshape: {query_vector.shape}\n')

  # Search the FAISS index
  distances, indices = index.search(query_vector, min(k, len(metadata)))
  print(f'The distances are {distances}\n')
  print(f'The indices are {indices}\n')

  # Store the indices and distances
  stored_indices = indices[0].tolist()
  stored_distances = distances[0].tolist()
  print(f'The stored indices are {stored_indices}\n')
  print(f'The stored distances are {stored_distances}\n')

  # Print everything (indices, distance, metadata) -> Debugging
  print(f'The metadata content is: \n')
  for i, dist in zip(stored_indices, stored_distances):
    if 0 <= i < len(metadata): # 0 <= i: Asegura que el índice no es negativo. i < len(metadata): Asegura que el índice no excede el tamaño total de la lista de metadatos.
      print(f'Distance: {dist}, \nMetadata: {metadata[i]['recipe_info']}')

  # Return results
  # Distancia con de la query con la info de la receta
  # Los results son guardados como tuplas en diccionarios
  results = [(
      metadata[i]['recipe_info'], dist
      )
  for i, dist in zip(stored_indices, stored_distances) if 0 <= i < len(metadata)]
  return results

¿Qué hace zip() exactamente?
zip() toma múltiples iterables (en este caso, dos listas) y devuelve un iterador de tuplas, donde cada tupla contiene un elemento de cada lista.1. El Objeto ZipSi tienes:stored_indices = [42, 105, 5]stored_distances = [0.012, 0.015, 0.021]zip(stored_indices, stored_distances) devuelve un iterador que, cuando se recorre, produce:$$(42, 0.012), (105, 0.015), (5, 0.021)$$

Output from query_embeddings:
Esto genera la siguiente estructura:$$\text{results} = [ (\text{String con texto de la receta 1}, \text{Float de Distancia 1}),$$$$\quad \quad \quad \quad (\text{String con texto de la receta 2}, \text{Float de Distancia 2}),$$$$\quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \dots ]$$

In [85]:
# Test the retrival system
query = 'How to make bread?'
results = query_embeddings(query, index, metadata)
print(f'The results are {results}')

The query embedding is [-0.019720381125807762, -0.028134725987911224, -0.022066915407776833, 0.016603518277406693, -0.04792621359229088, -0.04887430742383003, 0.038018617779016495, 0.017954552546143532, 0.0015480617294088006, 0.004168656188994646, -0.004832322709262371, -0.013889594934880733, -0.012917797081172466, -0.0015850967029109597, 0.038018617779016495, -0.0238090418279171, 0.017018308863043785, -0.016674624755978584, -0.0072233001701533794, 0.002310981974005699, -0.031856000423431396, 0.02015887387096882, 0.01501545775681734, -0.007821785286068916, 0.006275205407291651, 0.01784789189696312, 0.003410179866477847, -0.011229002848267555, -0.03960667923092842, 0.042285047471523285, -0.0036649806424975395, 0.023678677156567574, -0.034676581621170044, -0.02115437388420105, -0.015904298052191734, 0.011306035332381725, -0.01473102904856205, 0.005285630933940411, 0.00849730335175991, 0.019412249326705933, -0.01242004707455635, 0.007833636365830898, 0.00040108870598487556, -0.00825435388

video = Combining Outputs for Enhanced Results

In [86]:
# Combine the results
def combined_retrieved_content(results):
  combined_content = "\n\n".join([result[0] for result in results])
  return combined_content
print(f'The combined content is: {combined_retrieved_content}')

The combined content is: <function combined_retrieved_content at 0x7dbdf9e92e80>


La función combined_retrieved_content está diseñada específicamente para desechar el valor de la distancia y quedarse únicamente con el contenido textual, ya que el Modelo de Lenguaje Grande (LLM) sólo necesita el texto como contexto.

video = Constructing a Generative Model

# Generative System

In [87]:
# Define the system prompt
system_prompt3 = """
You are highly experinced and expert chef specialized in providing cooking advice.
Your main task is to provide information precise and accurate on the combined content.
You answer directly to the query using only information form the provided {combined_content}.
If you don't know the answer, just say that you don't know.
Your goal is to help the user and answer the {query}."""

In [88]:
# Define functionto retrieve from API
def generate_response(query, combined_content, system_prompt):
  response = client.chat.completions.create(
      model = model,
      messages = [
          {'role': 'system', 'content': system_prompt3},
          {'role':'user', 'content': query},
          {'role': 'assistant', 'content': combined_content}
      ],
      temperature = 0
  )
  return response

In [90]:
from IPython.display import Markdown, display

In [91]:
# Get the results
query = 'Get me the best chocolate cake recipe'
combined_content = combined_retrieved_content(results)
response = generate_response(query, combined_content, system_prompt3)
get_gpt_response()

I don't know.

video = Complete RAG System Implementation

# Rag System

In [92]:
# Build the function for RAG
def rag_system(query, index, metadata, system_prompt, k = 5):
  # Retrival system
  results = query_embeddings(query, index, metadata, k)

  # Content merge
  combined_content = combined_retrieved_content(results)

  # Generation
  response = generate_response(query, combined_content, system_prompt)

  # Return the response
  return response

In [96]:
# Test the rag system
query1 = 'How to make the best chocolate cake?'
response = rag_system(query1, index, metadata, system_prompt3)
get_gpt_response(response)

The query embedding is [-0.002604733919724822, -0.03968388959765434, -0.013297276571393013, 0.007617205381393433, -0.019327564164996147, 0.022370068356394768, 0.00810969714075327, -0.005398256704211235, -0.006457113660871983, 0.037473149597644806, -0.0003669747384265065, 0.007113769184798002, -0.05170068517327309, -0.007600788958370686, 0.0003343129646964371, -0.04031865671277046, 0.021166199818253517, -0.004692351911216974, -0.004487146623432636, -0.015858232975006104, 0.015004580840468407, -0.02479969523847103, -0.011732247658073902, 0.03399287164211273, 0.0030698650516569614, 0.006758080795407295, 0.022610843181610107, -0.0006980386096984148, -0.036904048174619675, 0.05677882209420204, 0.04460880532860756, 0.0018906210316345096, -0.016208449378609657, -0.014720030128955841, -0.02911173366010189, 0.016919827088713646, -0.004768961574882269, -0.001930293976329267, 0.03475897014141083, 0.014402646571397781, -0.005751208867877722, 5.8526144130155444e-05, 0.002621150342747569, 0.01079104

I don't know.

In [97]:
# Test with a different query
query2 = 'I want something vegan'
response = rag_system(query2, index, metadata, system_prompt3)
get_gpt_response(response)

The query embedding is [-0.034548308700323105, -0.026619188487529755, -0.017104245722293854, 0.03446335345506668, -0.01730247214436531, -0.008516724221408367, 5.26128314959351e-06, -0.018930774182081223, -0.033670444041490555, 0.023518336936831474, 0.003238904057070613, -0.012955616228282452, 0.0179679524153471, -0.007178685627877712, 0.036842089146375656, -0.008821146562695503, 0.009663615375757217, -0.01138395071029663, 0.007461868692189455, -0.018406886607408524, -0.023546654731035233, 0.00599993672221899, 0.037436775863170624, -0.0069061219692230225, 0.007702574133872986, 0.008198143914341927, -0.022074105218052864, -0.012240579351782799, 0.002099093049764633, 0.01846352219581604, 0.010208742693066597, 0.024481158703565598, -0.0050618937239050865, 0.05527729541063309, -0.013698970898985863, -0.003830048255622387, 0.038739416748285294, -0.0019114842871204019, 0.015617535449564457, 0.026930689811706543, -0.04777294769883156, 0.022003307938575745, -0.024226294830441475, 0.001365472329

Here are some vegan options based on the provided recipes:

1. **Brown Bread (Baked)**: This recipe can be made vegan by using water instead of milk and ensuring that the molasses is suitable for a vegan diet. The ingredients include Indian meal, rye meal, flour, water, and soda.

2. **Bannocks**: This recipe can also be adapted to be vegan by substituting the sour milk with a plant-based alternative (like almond milk or soy milk) and omitting any eggs if they are included in your version.

3. **Boston Brown Bread**: Similar to the brown bread, you can use plant-based milk instead of sour milk and ensure that the molasses is vegan-friendly.

These recipes focus on plant-based ingredients and can be adjusted to fit a vegan diet.